# Title

In [1]:
"""
    TITLE   :
    PROJECT :
"""

__author__ = ""
__version__ = ""

<span style='font-size:30px;font-weight:650'>
    About
</span>

information about this notebook


<br><br>

- - - 

## Prepare


### Imports

In [2]:
# THIRD PARTY
import agama
import astropy.coordinates as coord
import astropy.units as u
import numpy as np
from astropy.visualization import quantity_support
from galpy import df as gdf
from galpy import potential as gpot

# LOCAL
# PROJECT-SPECIFIC
from discO import GaussianMeasurementErrorSampler, PotentialFitter, PotentialSampler, conf
from discO.plugin.agama.fitter import AGAMAMultipolePotentialFitter, AGAMAPotentialFitter

In [3]:
quantity_support();

### Parameters

In [4]:
mass = 1e12 * u.solMass
r0 = 10 * u.kpc  # scale factor

<br><br>

- - - 

## Code

In [5]:
PotentialSampler._registry

{'agama': discO.plugin.agama.sample.AGAMAPotentialSampler,
 'galpy': discO.plugin.galpy.sample.GalpyPotentialSampler}

In [6]:
hernquist_pot = gpot.HernquistPotential(amp=mass, a=r0)
sampler = PotentialSampler(hernquist_pot)
sampler

In [7]:
sampler.sample(n=3, frame=coord.Galactocentric(), random=0)

<SkyCoord (Galactocentric: galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg): (x, y, z) in kpc
    [(27.1172228 ,  -7.89208216,   3.56928375),
     (30.0881722 ,  15.71499527, -42.94001442),
     ( 7.78532477, -10.31150342, -32.19463138)]
 (v_x, v_y, v_z) in km / s
    [( -23.43881734, -58.22560595, -125.31107118),
     (-134.65499462, -70.40401722,   59.07991269),
     (  66.70000353,  46.03324887, -142.03489184)]>

In [8]:
samples = sampler.sample(n=3, niter=2, frame=None, random=None)
samples

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [[(228.47394763, -23.44529675,   8.7744884 ),
      (139.71455856,  -6.33201948,  53.21459486)],
     [(141.4330694 , -58.20557827, 159.73499923),
      (257.48785022,  -8.48904108,  20.26954336)],
     [(106.80608422, -10.38488126,  32.16264965),
      (  9.15471126, -20.36140278,  28.59746488)]]
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    [[(-3.11262307e+00, -5.0451043 ,  -98.6587091 ),
      (-3.38050841e-01,  0.07845362,  267.57994542)],
     [( 4.24479960e-04,  0.15739561,  269.0456028 ),
      (-2.22841128e+00, -2.55825835, -286.27947762)],
     [(-4.86802296e-01, -1.84616346,  131.7489185 ),
      (-3.30005209e-01, -0.86782519,  178.30272477)]]>

In [9]:
meas = GaussianMeasurementErrorSampler(c_err=0.1)  # Note! not 10%, just 0.1

samps = sampler.sample(n=3, niter=2, frame=coord.Galactocentric(), random=0)

meas(samps)

<SkyCoord (Galactocentric: galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg): (x, y, z) in kpc
    [[(27.2533559 ,  -7.99571288,   3.62182835),
      (30.2343536 ,  15.84875145, -43.02157771)],
     [( 7.75435465, -10.41796935, -32.17488259),
      (27.07607157,  -7.78650818,   3.57517316)],
     [(30.14487454,  15.79686217, -42.93728097),
      ( 7.56685117, -10.24302387, -32.36044107)]]>

AGAMA

In [10]:
pot_Hern = agama.Potential(
    type="Spheroid",
    mass=1e12,
    scaleRadius=10,
    gamma=1,
    alpha=1,
    beta=4,
    cutoffStrength=0,
)

In [11]:
sampler = PotentialSampler(
    agama.Potential(
        type="Spheroid",
        mass=1e12,
        scaleRadius=10,
        gamma=1,
        alpha=1,
        beta=4,
        cutoffStrength=0,
    ),
)

sampler

In [12]:
samples = sampler.sample(n=10)
samples

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [(269.77768587, -80.39242629,  12.15818053),
     (211.53383512,  -3.67881258,   7.37721302),
     (135.4959591 , -44.62636438, 156.25820005),
     (  3.85684546,  -7.46999137,   5.08874191),
     ( 42.11155402, -20.90390085,   7.7856392 ),
     (212.56246467, -64.15957604,  16.58761413),
     (203.11313493,  -9.16456976,   6.31741618),
     ( 61.49015059, -33.66474899,   3.83061213),
     (344.11597718, -41.05292432,   4.97326983),
     ( 98.63435577,  16.56923216,  32.21408322)]>

In [13]:
array = sampler.sample(n=10, niter=20)
array[:1, :3]

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [[(132.33728927,  63.57520897,  17.45039135),
      (217.32849778, -80.57852946,   2.6643645 ),
      (353.59709381,  47.89216648, 174.37658588)]]>

In [14]:
meas = GaussianMeasurementErrorSampler(c_err=0.1)

sample = sampler.sample(n=10)
meas(sample)

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [(300.28094958, -37.01741098,  20.09047913),
     (107.00778556,   8.26602531, 157.55744141),
     ( 79.23205649,  23.97670472,  72.83514259),
     (311.841145  , -29.46958289,  29.55753177),
     (256.00773015, -32.7563284 ,   4.77082656),
     (269.59643138, -32.44097596, 145.78864957),
     (142.42449445,  50.42324959, 108.43387374),
     (158.26554746, -26.7407235 ,  54.30242495),
     (296.73301622,  73.95095757,   2.53676585),
     (263.8464258 ,  -0.1689742 ,  20.91166967)]>

In [15]:
PotentialFitter[("agama", "multipole")] == PotentialFitter["agama"][
    "multipole"
] == AGAMAPotentialFitter["multipole"]

fitter = PotentialFitter[("agama", "multipole")]()
fitter

In [16]:
fit = AGAMAMultipolePotentialFitter(symmetry="a")(sample)
fit

In [20]:
fitter.fit(array).shape

(20,)

---

Now pipe with fitters

In [18]:
# sampler = PotentialSampler(gdf.isotropicHernquistdf(hernquist_pot))
# fitter = PotentialFitter(
#     "multipole", package="agama", return_specific_class=True
# )
# ms = GaussianMeasurementErrorSampler()

# pipe = sampler | ms | fitter

# fit_pot = pipe(n=10000)

# fit_pot

In [19]:
# ms(sampler.sample(n=3), c_err=0.01)

<br><br>

- - - 

<span style='font-size:40px;font-weight:650'>
    END
</span>